In [3]:
(require '[clojupyter.misc.helper :as helper])
(helper/add-dependencies '[libpython-clj "1.6-SNAPSHOT"])

{[org.clojure/core.memoize "0.7.2"] nil, [org.apache.commons/commons-math3 "3.6.1"] nil, [com.tdunning/t-digest "3.2"] nil, [techascent/tech.jna "3.18"] nil, [techascent/tech.parallel "2.1"] nil, [camel-snake-kebab "0.4.0"] nil, [org.clojure/tools.analyzer "0.6.9"] nil, [org.clojure/tools.reader "1.0.0-beta4"] nil, [org.clojure/core.async "0.4.500"] nil, [net.java.dev.jna/jna "5.2.0"] nil, [it.unimi.dsi/fastutil "8.2.1"] nil, [libpython-clj "1.6-20191102.075123-1"] #{[camel-snake-kebab "0.4.0"] [techascent/tech.datatype "4.25"] [org.clojure/clojure "1.10.1"]}, [org.clojure/math.combinatorics "0.1.4"] nil, [org.clojure/test.check "0.9.0"] nil, [techascent/tech.resource "4.5"] nil, [org.clojure/core.cache "0.7.1"] nil, [org.clojure/tools.logging "0.4.1"] nil, [org.ow2.asm/asm-all "4.2"] nil, [org.clojure/data.priority-map "0.0.7"] nil, [techascent/tech.datatype "4.25"] #{[org.clojure/core.memoize "0.7.2"] [org.apache.commons/commons-math3 "3.6.1"] [com.tdunning/t-digest "3.2"] [techascen

In [4]:
(ns tensorcloj.core-draft
  ""
  (:require [libpython-clj.python
             :refer [import-module
                     ->py-tuple
                     get-item
                     get-attr
                     python-type
                     call-attr
                     call-attr-kw
                     att-type-map
                     ->py-dict]
             :as py]
            [clojure.pprint :as pp]))

nil

In [5]:
(py/initialize!)

:ok

In [6]:
(defonce tf (import-module "tensorflow"))
(defonce keras (import-module "keras"))
(defonce keras-layers (import-module "keras.layers"))
(defonce keras-models (import-module "keras.models"))
;Helper libraries
(defonce np (import-module "numpy"))

Using TensorFlow backend.


#'tensorcloj.core-draft/np

In [7]:
(defonce fashionmnist (import-module "keras.datasets.fashion_mnist"))
(def mnist-data (py/call-attr fashionmnist "load_data"))


32768/29515 [=================================] - 0s 2us/step

14671872/26421880 [===============>..............] - ETA: 6s


26427392/26421880 [==============================] - 13s 0us/step

8192/5148 [===============================================] - 0s 0us/step

4423680/4422102 [==============================] - 2s 0us/step


#'tensorcloj.core-draft/mnist-data

In [8]:
(defmacro def+
  "binding => binding-form
  internalizes binding-forms as if by def."
  {:added "1.9", :special-form true, :forms '[(def+ [bindings*])]}
  [& bindings]
  (let [bings (partition 2 (destructure bindings))]
    (sequence cat
              ['(do)
               (map (fn [[var value]] `(def ~var ~value)) bings)
               [(mapv (fn [[var _]] (str var)) bings)]])))

#'tensorcloj.core-draft/def+

In [9]:
(def+ [[train-images , train-labels], [test-images, test-labels]] (vec mnist-data))

["vec__28656" "vec__28659" "train-images" "train-labels" "vec__28662" "test-images" "test-labels"]

In [10]:
(def class-name  ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat"
                  "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"])

#'tensorcloj.core-draft/class-name

In [15]:
(def model (py/call-attr  keras "Sequential" [
                           (py/call-attr-kw keras-layers "Flatten" [] {:input_shape   [28,28]})
                           (py/call-attr-kw keras-layers "Dense" [128] {:activation  "relu"})
                           (py/call-attr-kw keras-layers "Dense" [10] {:activation "softmax" })]))

#'tensorcloj.core-draft/model

In [16]:
(py/call-attr-kw  model "compile" [] {:optimizer "adam"
               :loss "sparse_categorical_crossentropy"
               :metrics (py/->py-list["accuracy"])})

nil

In [17]:
(py/call-attr-kw model "fit" [train-images, train-labels ] {:epochs 10})


Epoch 1/10

46016/60000 [======================>.......] - ETA: 1s - loss: 4.2592 - accuracy: 0.6885


60000/60000 [==============================] - 7s 118us/step - loss: 3.4658 - accuracy: 0.6935
Epoch 2/10

30464/60000 [==============>...............] - ETA: 3s - loss: 0.7804 - accuracy: 0.7337


60000/60000 [==============================] - 6s 101us/step - loss: 0.7054 - accuracy: 0.7569
Epoch 3/10

19200/60000 [========>.....................] - ETA: 3s - loss: 0.5933 - accuracy: 0.7949


60000/60000 [==============================] - 6s 98us/step - loss: 0.5669 - accuracy: 0.8026
Epoch 4/10

 4448/60000 [=>............................] - ETA: 5s - loss: 0.4892 - accuracy: 0.8170


51776/60000 [========================>.....] - ETA: 0s - loss: 0.5330 - accuracy: 0.8112


...

60000/60000 [==============================] - 7s 109us/step - loss: 0.4786 - accuracy: 0.8342
Epoch 8/10

38016/60000 [==================>...........] - ETA: 2s - loss: 0.4613 - accuracy: 0.8429


60000/60000 [==============================] - 6s 107us/step - loss: 0.4716 - accuracy: 0.8389
Epoch 9/10

21920/60000 [=========>....................] - ETA: 4s - loss: 0.4610 - accuracy: 0.8406


60000/60000 [==============================] - 7s 109us/step - loss: 0.4675 - accuracy: 0.8400
Epoch 10/10

 5472/60000 [=>............................] - ETA: 5s - loss: 0.4519 - accuracy: 0.8489


52128/60000 [=========================>....] - ETA: 0s - loss: 0.4700 - accuracy: 0.8410


60000/60000 [==============================] - 6s 102us/step - loss: 0.4680 - accuracy: 0.8414


In [18]:
(def+ [test-loss test-acc]  (py/call-attr-kw model "evaluate" [test-images, test-labels ] {:verbose 2} ) )


["vec__28673" "test-loss" "test-acc"]

In [19]:
(def predictions (py/call-attr model "predict" test-images))

#'tensorcloj.core-draft/predictions

In [20]:
(py/call-attr np "argmax" (first predictions))

9

In [21]:
(first test-labels)

9